<a href="https://colab.research.google.com/github/Jrodos12/Organizacion-de-Datos/blob/main/Analisis%20exploratorio%20de%20datos/spark21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#El contenido con mayor cantidad de de acciones realizadas para todos los tipos posibles de acciones (⭐⭐⭐).

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=af8e2437583e6d5684000163470ffc3a93ea4a538e00631d65fb459d973e2790
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import datetime as dt

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext


In [ ]:
sqlContext = SQLContext(sc)
df_logs = sqlContext.read.csv("drive/MyDrive/logs.csv", header=True, inferSchema=True)
rdd_logs = df_logs.select("action", "title").rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd = rdd_logs.map(lambda x: ( x.action,(x.action, x.title)) )

In [ ]:
acciones = ['delete', 'block', 'upload', 'protect', 'restore', 'unblock',
       'unprotect', 'rights', 'move', 'move_redir', 'renameuser',
       'newusers', 'create', 'create2', 'modify', 'patrol', 'autocreate',
       'usergroups', 'move_prot', 'reblock', 'event', 'revision',
       'interwiki', 'byemail', 'thank', 'add', 'update', 'remove',
       'selfadd', 'selfremove', 'review', 'failure', 'skipbadns', 'send',
       'unreview', 'skipnouser', 'profilesave', 'undelete', 'merge',
       'mergeuser', 'deleteuser', 'delete_redir', 'whitelist', 'change',
       'new', 'addlink']

filtro por acciones posibles ya que hay varios post donde se subio como accion el titulo u otra cosa en vez de la accion perse, por lo tanto no me sirve.
Ademas filtro los titulos nulos, ya que estos no tienen posibilidad de relacionarse

In [ ]:
rdd = rdd.filter(lambda x: x[0] in acciones and x[1][1] != None)

In [ ]:
repeticiones = {}
def contar(x):
  repeticiones[x[1]] = repeticiones.get(x[1],0) + 1
  return repeticiones[x[1]]
df = rdd.map(lambda x: (x[0], (x[1][1], contar(x))) )

In [ ]:
df.reduceByKey(lambda x,y: x if x[1] > y[1] else y).collect()

[('selfremove',
  ('Education Program:Wikimedia Chile/Análisis de Información Documental I (2016-1)',
   5)),
 ('send',
  ('Wikipedia:Candidatos a artículos destacados/Boletín/Lista/Prueba', 4)),
 ('autocreate', ('Usuario:MaSt', 3)),
 ('usergroups', ('Usuario:Dferg', 3)),
 ('interwiki', ('Actinozoa', 2)),
 ('block', ('Usuario:Antoine', 32)),
 ('move_prot', ('Plantilla:Ficha de entidad subnacional', 34)),
 ('selfadd',
  ('Education Program:Tec de Monterrey, Campus Ciudad de Mexico/Experiencias Retadoras CCM (ene-mayo 2015)',
   163)),
 ('skipbadns', ('User Calypso', 13)),
 ('profilesave', ('Especial:PerfilDeVoluntarioEnLínea', 6)),
 ('mergeuser', ('Usuario:DerHexer', 1)),
 ('renameuser', ('Especial:Renameuser', 102)),
 ('update', ('Wikipedia:Portada', 505)),
 ('remove',
  ('Education Program:Tec de Monterrey, Campus Ciudad de Mexico/Experiencias Retadoras CCM (ene-mayo 2015)',
   10)),
 ('review', ('Ingeniería de software', 13)),
 ('thank', ('Usuario:Ontzak', 1486)),
 ('add', ('(commons